In [24]:
%pip install torch
%pip install transformers
%pip install ipywidgets
%pip install datasets


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-macosx_10_13_universal2.whl.metadata (37 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
 

In [40]:
import torch
import transformers
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from itertools import product
from collections import defaultdict

pd.set_option('display.width',1000)


In [41]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

MAX_LENGTH = 128
K = 5

LR = 2e-5
EPOCHS = 3
BSZ = 16

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [ ]:
from dont_patronize_me import DontPatronizeMe

DATA_PATH = 'data/'

dpm = DontPatronizeMe(DATA_PATH, DATA_PATH)
dpm.load_task1()

data = dpm.train_task1_df

trids = pd.read_csv(f'{DATA_PATH}train_semeval_parids-labels.csv')
teids = pd.read_csv(f'{DATA_PATH}dev_semeval_parids-labels.csv')

trids['par_id'] = trids.par_id.astype(str)
teids['par_id'] = teids.par_id.astype(str)

cols = ['par_id', 'text', 'label_y']

trdf = trids.merge(data, on='par_id', how='left')[cols]
tedf = teids.merge(data, on='par_id', how='left')[cols]

print(trdf.head())
print()
print(tedf.head())


  par_id                                               text  label_y
0   4341  The scheme saw an estimated 150,000 children f...        1
1   4136  Durban 's homeless communities reconciliation ...        1
2  10352  The next immediate problem that cropped up was...        1
3   8279  Far more important than the implications for t...        1
4   1164  To strengthen child-sensitive social protectio...        1

  par_id                                               text  label_y
0   4046  We also know that they can benefit by receivin...        1
1   1279  Pope Francis washed and kissed the feet of Mus...        1
2   8330  Many refugees do n't want to be resettled anyw...        1
3   4063  "Budding chefs , like "" Fred "" , "" Winston ...        1
4   4089  "In a 90-degree view of his constituency , one...        1


In [33]:
counts = trdf['label_y'].value_counts()
print("Original class distribution:")
print(counts)
print()

weights = torch.tensor([1.0 / counts[0], 1.0 / counts[1]]).to(device)

trds = Dataset.from_pandas(trdf)
print(trds)

# # Upsample positive class
# pos_df = trdf[trdf['label_y'] == 1]
# neg_df = trdf[trdf['label_y'] == 0]

# upsampled_pos_df = pos_df.sample(n=len(neg_df), replace=True, random_state=RANDOM_SEED)
# trdf_balanced = pd.concat([neg_df, upsampled_pos_df]).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# print("\nBalanced class distribution:")
# print(trdf_balanced['label_y'].value_counts())

Original class distribution:
label_y
0    7581
1     794
Name: count, dtype: int64

Dataset({
    features: ['par_id', 'text', 'label_y'],
    num_rows: 8375
})


In [37]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=RANDOM_SEED)

X = trdf["text"].values
y = trdf["label_y"].values

folds = list(skf.split(X, y))


In [34]:
class BalancedTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop("label_y").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, self.model.config.num_labels)

        loss_fn = nn.CrossEntropyLoss(weight=weights)

        return loss_fn(logits, labels)


In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


rob_checkpoint = "FacebookAI/roberta-base"
deb_checkpoint = "microsoft/deberta-base"

rob_tokenizer = AutoTokenizer.from_pretrained(rob_checkpoint)
deb_tokenizer = AutoTokenizer.from_pretrained(deb_checkpoint)

def tokenize(tokenizer, batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

architectures = [("Roberta", rob_checkpoint, rob_tokenizer), ("Deberta", deb_checkpoint, deb_tokenizer)]


In [ ]:
LRS = [2e-5, 3e-5, 5e-5]
BSZS = [16, 32]

# Determine best hyperparameters using k-fold cross-validation
def hyperparameter_search():
    results = {}
    results["Roberta"] = defaultdict(float)
    results["Deberta"] = defaultdict(float)

    for name, checkpoint, tokenizer in architectures:
        print(f"Training {name}...")
        print()

        for fold, (train_idx, val_idx) in enumerate(folds):
            print(f"Fold {fold + 1}/{K}")

            trds = Dataset.from_pandas(trdf.iloc[train_idx])
            valds = Dataset.from_pandas(trdf.iloc[val_idx])

            trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
            valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

            trds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])
            valds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])

            for lr, bsz in product(LRS, BSZS):
                model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)
                training_args = TrainingArguments(
                    output_dir=f'./results/fold_{fold+1}/{name}_lr{lr}_bsz{bsz}',
                    learning_rate=lr,
                    per_device_train_batch_size=bsz,
                    per_device_eval_batch_size=bsz,
                    num_train_epochs=EPOCHS,
                    logging_dir=f'./logs/fold_{fold+1}/{name}_lr{lr}_bsz{bsz}',
                    save_strategy="epoch"
                )

                trainer = BalancedTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=trds,
                    eval_dataset=valds
                )

                trainer.train()
                metrics = trainer.evaluate()

                results[name][(lr, bsz)] += metrics["eval_f1"]

    best_hyperparams = {
        name: max(results[name], key=results[name].get)
        for name in results
    }

    return best_hyperparams

In [48]:
# Retrain best models on full training set
def train_models(best_hyperparams):
    models = {}

    for name, checkpoint, tokenizer in architectures:
        print(f"Retraining {name} on full training set...")

        trds = Dataset.from_pandas(trdf)
        valds = Dataset.from_pandas(tedf)

        trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

        trds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])
        valds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])

        lr, bsz = best_hyperparams[name]

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)
        training_args = TrainingArguments(
            output_dir=f'./final_results/{name}',
            learning_rate=lr,
            per_device_train_batch_size=bsz,
            per_device_eval_batch_size=bsz,
            num_train_epochs=EPOCHS,
            logging_dir=f'./final_logs/{name}',
            save_strategy="epoch"
        )

        trainer = BalancedTrainer(
            model=model,
            args=training_args,
            train_dataset=trds,
            eval_dataset=valds
        )

        trainer.train()
        metrics = trainer.evaluate()

        models[name] = (model, tokenizer, metrics["eval_f1"])
    
    # Calculate relative F1 scores for ensembling
    tot_f1 = sum(m[2] for m in models.values())
    for name in models:
        model, tokenizer, f1 = models[name]

        models[name] = (model, tokenizer, f1 / tot_f1)

    return models
